In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# New Section

In [ ]:
data = pd.read_csv('sonar.all-data.csv')

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
print("Total Columns : ", len(data.columns))

In [ ]:
data[data.columns[60]].value_counts()

In [ ]:
X = data[data.columns[0:60]].values
y = data[data.columns[60]].values

In [ ]:
print(X.shape)

In [ ]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [ ]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
Y = one_hot_encode(y)

In [ ]:
Y[0]

In [ ]:
X,Y = shuffle (X, Y, random_state = 0)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [ ]:
learning_rate = 0.2
training_epochs = 500
n_dim = X.shape[1]
print("n_dim = ", n_dim)
n_class = 2

In [ ]:
cost_history = np.empty(shape=[1],dtype=float)

In [ ]:
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60

In [ ]:
x = tf.compat.v1.placeholder(tf.float32, [None, n_dim])
W = tf.Variable(tf.zeros([n_dim, n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.compat.v1.placeholder(tf.float32, [None, n_class])

In [ ]:
def multilayer_perceptron(x, weights, biases):

    # Hidden layer with RELU activationsd
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

    # Hidden layer with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Hidden layer with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)

    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.sigmoid(layer_4)

    # Output layer with linear activation
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    return out_layer

# Define the weights for each layers

In [ ]:
# Define the weights for each layers

weights = {
    'h1': tf.Variable(tf.compat.v1.truncated_normal([n_dim, n_hidden_1])),
    'h2': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_4, n_class]))
}

In [ ]:

biases = {
    'b1': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_1])),
    'b2': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_2])),
    'b3': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_3])),
    'b4': tf.Variable(tf.compat.v1.truncated_normal([n_hidden_4])),
    'out': tf.Variable(tf.compat.v1.truncated_normal([n_class]))
}

In [ ]:
init = tf.compat.v1.global_variables_initializer()

In [ ]:
y = multilayer_perceptron(x, weights, biases)

In [ ]:
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
training_step = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [ ]:
sess = tf.compat.v1.Session()
sess.run(init)

In [ ]:
mse_history = []
accuracy_history = []

In [ ]:
for epoch in range(training_epochs):
    sess.run(training_step, feed_dict = {x: train_x, y_: train_y})
    cost = sess.run(cost_function, feed_dict={x: train_x, y_: train_y})
    cost_history = np.append(cost_history, cost)
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    pred_y = sess.run(y, feed_dict = {x: test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy, feed_dict={x: train_x, y_: train_y}))
    accuracy_history.append(accuracy)

    if epoch % 50 == 0:
        print('epoch : ', epoch, ' ; ', 'cost: ', cost, " ; MSE: ", mse_, "- Train Accuracy: ", accuracy )

epoch :  150  ;  cost:  1.4088273  ; MSE:  4.971607517200296 - Train Accuracy:  0.54545456
epoch :  200  ;  cost:  1.3552343  ; MSE:  6.15817169280856 - Train Accuracy:  0.58181816
epoch :  250  ;  cost:  0.57314384  ; MSE:  2.7474274348986114 - Train Accuracy:  0.77575755
epoch :  300  ;  cost:  0.43428013  ; MSE:  5.095224158775006 - Train Accuracy:  0.77575755
epoch :  350  ;  cost:  0.27691722  ; MSE:  4.0257882057790795 - Train Accuracy:  0.8848485
epoch :  400  ;  cost:  0.17262216  ; MSE:  4.449286378469919 - Train Accuracy:  0.95757574
epoch :  450  ;  cost:  0.108565316  ; MSE:  5.5417108000166975 - Train Accuracy:  0.9757576


In [ ]:
plt.plot(range(len(cost_history)), cost_history)
plt.xlabel('Epochs ')
plt.ylabel('Cost_History ')
plt.title("Cost per Epoch Graph")
plt.xlim(0, training_epochs + 10)
plt.ylim(0, np.max(cost_history))
plt.rcParams['figure.figsize'] = [12, 5]
plt.show()

In [ ]:
plt.plot(range(len(accuracy_history)), accuracy_history)
plt.xlabel('Epochs ')
plt.ylabel('Accuracy_History ')
plt.title("Cost per Epoch Graph")
plt.xlim(0, training_epochs + 10)
plt.ylim(0.4, np.max(accuracy_history)+0.2)
plt.rcParams['figure.figsize'] = [12, 5]
plt.show()

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Test Accuracy: ", (sess.run(accuracy, feed_dict={x: test_x, y_: test_y})))

In [ ]:
pred_y = sess.run(y, feed_dict={x: test_x})
mse = tf.reduce_mean(tf.square(pred_y - test_y))
print("MSE: %.4f" % sess.run(mse))